# Word Embedding
https://www.youtube.com/watch?v=5PL0TmQhItY

In [34]:
!pip install torch nltk pandas seaborn chardet

In [24]:
import nltk
import pandas as pd
import seaborn as sns
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [36]:
from google.colab import drive
drive.mount('/content/gdrive')
file_name = "gdrive/MyDrive/tweets.csv"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [66]:
from chardet.universaldetector import UniversalDetector

detector = UniversalDetector()
with open(file_name, "rb") as f:
  for line in f.readlines()[13300:35400]:
    detector.feed(line)
    if detector.done:
      break
detector.close()

print(detector.result)

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


In [67]:
tweets_df = pd.read_csv(file_name,  header=None, encoding='ISO-8859-1', names=["target", "ids", "date", "flag", "user", "text"])

In [68]:
tweets_df.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


# Word2Vec implementation

# Matrix Decomposition

# Glove